In [9]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [10]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [11]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

##### First step: Have GPT-4o-mini figure out which links are relevant
##### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

In [12]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [13]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}



In [14]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [15]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [16]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/microsoft/VibeVoice-1.5B',
 '/xai-org/grok-2',
 '/openbmb/MiniCPM-V-4_5',
 '/Qwen/Qwen-Image-Edit',
 '/deepseek-ai/DeepSeek-V3.1',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/zerogpu-aoti/wan2-2-fp8da-aoti-faster',
 '/spaces/syncora/synthetic-generation',
 '/spaces/multimodalart/Qwen-Image-Edit-Fast',
 '/spaces/Qwen/Qwen-Image-Edit',
 '/spaces',
 '/datasets/syncora/developer-productivity-simulated-behavioral-data',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/syncora/synthetic-healthcare-admissions',
 '/datasets/openai/healthbench',
 '/datasets/nvidia/Nemotron-Post-Training-Dataset-v2',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/m

In [17]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

##### Second step: Make the brochure

Assemble all the details into another gtp4-o prompt

In [18]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [20]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [21]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [22]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmicrosoft/VibeVoice-1.5B\nUpdated\n3 days ago\n•\n67.6k\n•\n1.07k\nxai-org/grok-2\nUpdated\n7 days ago\n•\n3.99k\n•\n862\nopenbmb/MiniCPM-V-4_5\nUpdated\nabout 13 hours ago\n•\n8.46k\n•\n725\nQwen/Qwen-Image-Edit\nUpdated\n6 days ago\n•\n72.3k\n•\n1.53k\ndeepseek-ai/DeepSeek-V3.1\nUpdated\n4 days ago\n•\n72.4k\n•\n660\nBrowse 1M+ models\nSpaces\nRunning\n12.5k\n12.5k\nDeepSite v2\n🐳\nGener

In [23]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [24]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'company blog', 'url': 'https://huggingface.co/blog'}]}


# Hugging Face Company Brochure

## Welcome to Hugging Face

At Hugging Face, we are **more than just a tech company**; we are a vibrant community dedicated to the development and democratization of artificial intelligence. Our mission is to shape the future of AI by providing a collaborative platform where individuals, organizations, and researchers can connect, innovate, and thrive. 

### What We Offer

- **Models**: With over **1 million models** available, users can explore machine learning solutions for a wide range of applications. From state-of-the-art AI models for text, image, and video to specialized datasets, our variety is expansive.

- **Datasets**: We host an extensive library of **250,000+ datasets**, enabling users to enhance their machine learning projects with high-quality data tailored for specific tasks.

- **Spaces**: Hugging Face offers over **400,000 applications** that facilitate real-time collaboration on projects, allowing users to build and showcase their work effortlessly.

### Our Customers

Hugging Face is proud to serve a diverse base of over **50,000 organizations** worldwide, including prominent names like Amazon, Google, Microsoft, and more. Our platform supports both non-profit and enterprise-level users, providing them with tools to enhance their AI capabilities.

### Company Culture

At Hugging Face, we embrace an **open-source philosophy**. We believe in the power of community collaboration to drive innovation in machine learning. Our environment fosters **creativity, collaboration, and continuous learning**, ensuring that each team member feels valued and empowered.

- **Inclusivity**: We prioritize creating an inclusive workplace that welcomes diverse perspectives.
- **Team Support**: Our focus on teamwork and support enables continuous personal and professional growth.
  
### Careers and Opportunities

Join us at Hugging Face to be part of an exciting journey where you can **make a real impact** in the AI landscape! We are on the lookout for passionate individuals in various roles—from machine learning engineers to community managers. Whether you are experienced or just starting your career, there’s a place for you in our team.

#### Benefits of Working With Us:

- **Flexible Work Environment**: We value work-life balance and offer remote working options.
- **Opportunities for Growth**: Continuous learning and development are at the core of our employee experience.
- **Community Engagement**: Work with a team that’s committed to reshaping the AI landscape for everyone.

### Join Us

Explore how you can be a part of the **Hugging Face** mission! To learn more about our offerings or apply for jobs, visit our website [Hugging Face](https://huggingface.co).

---

**Connect with Us!**
- Twitter: [@huggingface](https://twitter.com/huggingface)
- LinkedIn: [Hugging Face](https://www.linkedin.com/company/huggingface/)
- Discord: Join our community for live discussions and ideas!

**Hugging Face – The AI community building the future.**